![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# Classificação de Atividade Humana com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [11]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, plot_confusion_matrix
import time
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [8]:
# Substitua pelo caminho correto
base_path = "/Users/samwalford/Downloads/UCI HAR Dataset 3/"

# Atualizando os caminhos para os arquivos
filename_features = base_path + "features.txt"
filename_labels = base_path + "activity_labels.txt"

filename_subtrain = base_path + "train/subject_train.txt"
filename_xtrain = base_path + "train/X_train.txt"
filename_ytrain = base_path + "train/y_train.txt"

filename_subtest = base_path + "test/subject_test.txt"
filename_xtest = base_path + "test/X_test.txt"
filename_ytest = base_path + "test/y_test.txt"

# Carregando os arquivos
features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze()
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze()
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze()
X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

# Exibindo informações sobre os dados carregados
print("Features carregadas:", features.shape)
print("Labels carregadas:", labels.shape)
print("Treino:", X_train.shape, y_train.shape, subject_train.shape)
print("Teste:", X_test.shape, y_test.shape, subject_test.shape)


Features carregadas: (561,)
Labels carregadas: (6, 2)
Treino: (7352, 561) (7352, 1) (7352,)
Teste: (2947, 561) (2947, 1) (2947,)


## PCA com variáveis padronizadas

Reflexão sobre a escala das variáveis:

**Variáveis em métricas muito diferentes** podem interferir na análise de componentes principais. Lembra que variância é informação pra nós? Pois bem, tipicamente se há uma variável monetária como salário, vai ter uma ordem de variabilidade bem maior que número de filhos, tempo de emprego ou qualquer variável dummy. Assim, as variáveis de maior variância tendem a "dominar" a análise. Nesses casos é comum usar a padronização das variáveis.

Faça duas análises de componentes principais para a base do HAR - com e sem padronização e compare:

- A variância explicada por componente
- A variância explicada acumulada por componente
- A variância percentual por componente
- A variância percentual acumulada por componente
- Quantas componentes você escolheria, em cada caso para explicar 90% da variância?

## Análise de Componentes Principais (PCA) - Com e Sem Padronização

Nesta análise, realizamos duas abordagens distintas para o cálculo do PCA: uma sem padronização dos dados e outra com padronização. A seguir, comparamos os resultados em relação à variância explicada por componente, variância acumulada e a escolha de componentes para explicar 90% da variância.

### 1. PCA Sem Padronização

Sem padronização, os dados originais preservam suas escalas, o que pode resultar em uma dominância de variáveis com valores maiores na determinação das componentes principais.

**Resultados:**
- **Variância Explicada por Componente:** As primeiras componentes explicam a maior parte da variância, devido à influência de variáveis com escalas maiores.
- **Variância Explicada Acumulada:** A variância acumulada cresce rapidamente nas primeiras componentes, mas é influenciada desproporcionalmente pelas variáveis dominantes.
- **Variância Percentual por Componente:** Mostra o percentual de variância capturado por cada componente principal.
- **Variância Percentual Acumulada:** A soma dos percentuais de variância explicada até cada componente.

**Escolha de Componentes:**
Para explicar **90% da variância**, utilizamos aproximadamente _N componentes (resultado específico baseado no cálculo)_.

---

### 2. PCA Com Padronização

Com padronização, todas as variáveis têm média 0 e desvio padrão 1, garantindo que cada variável contribua igualmente para o cálculo do PCA.

**Resultados:**
- **Variância Explicada por Componente:** A distribuição da variância é mais uniforme, pois nenhuma variável domina as outras.
- **Variância Explicada Acumulada:** A variância acumulada aumenta de forma mais gradual e consistente.
- **Variância Percentual por Componente:** Representa a contribuição percentual de cada componente.
- **Variância Percentual Acumulada:** Fornece a soma das variâncias percentuais até cada componente.

**Escolha de Componentes:**
Para explicar **90% da variância**, utilizamos aproximadamente _M componentes (resultado específico baseado no cálculo)_.

---

### 3. Comparação dos Resultados

| Métrica                          | Sem Padronização       | Com Padronização         |
|----------------------------------|------------------------|--------------------------|
| Variância Explicada (Primeira Componente) | _X% (maior)_         | _Y% (menor, mais uniforme)_ |
| Variância Explicada Acumulada (90%)      | _N Componentes_       | _M Componentes_          |
| Contribuição das Variáveis      | Dominância de escalas  | Contribuição igualitária |

### 4. Conclusão

- **Padronização Recomendada:** A padronização é preferível, pois elimina o viés causado pelas escalas das variáveis.
- **Componentes para Explicar 90% da Variância:**
  - Sem Padronização: _N Componentes_
  - Com Padronização: _M Componentes_

A análise com padronização resulta em uma distribuição mais uniforme da variância entre as componentes principais, enquanto a análise sem padronização tende a ser influenciada desproporcionalmente por variáveis com maior escala.


In [10]:
# Função para padronizar os dados
def padroniza(s):
    if s.std() > 0:
        s = (s - s.mean()) / s.std()
    return s

# Padronizando os dados de treino e teste
start_time = time.time()
X_train_pad = pd.DataFrame(X_train).apply(padroniza, axis=0)
end_time = time.time()

print(f"Tempo de padronização: {end_time - start_time:.4f} segundos")

# PCA sem padronização
pca_no_scaling = PCA()
pca_no_scaling.fit(X_train)

# PCA com padronização
pca_with_scaling = PCA()
pca_with_scaling.fit(X_train_pad)

# Resumo dos resultados
components_no_scaling = (pca_no_scaling.explained_variance_ratio_.cumsum() < 0.90).sum() + 1
components_with_scaling = (pca_with_scaling.explained_variance_ratio_.cumsum() < 0.90).sum() + 1

print("\nResumo dos Resultados:")
print(f"- Componentes para 90% da variância (Sem Padronização): {components_no_scaling}")
print(f"- Componentes para 90% da variância (Com Padronização): {components_with_scaling}")



Tempo de padronização: 0.1906 segundos

Resumo dos Resultados:
- Componentes para 90% da variância (Sem Padronização): 34
- Componentes para 90% da variância (Com Padronização): 63


### Comparação entre PCA com e sem Padronização

A análise de componentes principais (PCA) revela diferenças importantes entre os dados com e sem padronização. 

- **Sem Padronização:** Apenas **34 componentes** são necessários para explicar 90% da variância. Isso ocorre porque a variância nos dados originais está altamente concentrada em algumas dimensões devido à diferença de escala entre as variáveis.
- **Com Padronização:** São necessários **63 componentes** para alcançar 90% da variância explicada. Isso acontece porque a padronização ajusta a escala das variáveis, distribuindo melhor a variância entre elas e permitindo que mais componentes contribuam para a explicação da variabilidade total.

Portanto, a padronização é essencial quando as variáveis possuem escalas diferentes, pois evita que algumas dominem a análise.


## Árvore com PCA

Faça duas uma árvore de decisão com 10 componentes principais - uma com base em dados padronizados e outra sem padronizar. Utilize o ```ccp_alpha=0.001```.

Compare a acurácia na base de treino e teste.

In [12]:
# Assumindo que X_train, X_test, y_train, y_test já estão definidos

# 1. Aplicar PCA com 10 componentes
def aplicar_pca(X_train, X_test, padronizar=False):
    if padronizar:
        X_train = pd.DataFrame(X_train).apply(lambda s: (s - s.mean()) / s.std(), axis=0)
        X_test = pd.DataFrame(X_test).apply(lambda s: (s - s.mean()) / s.std(), axis=0)
    pca = PCA(n_components=10)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca

# Dados com padronização
X_train_pca_pad, X_test_pca_pad = aplicar_pca(X_train, X_test, padronizar=True)

# Dados sem padronização
X_train_pca_no_pad, X_test_pca_no_pad = aplicar_pca(X_train, X_test, padronizar=False)

# 2. Criar árvores de decisão
tree_pad = DecisionTreeClassifier(ccp_alpha=0.001)
tree_no_pad = DecisionTreeClassifier(ccp_alpha=0.001)

# Treinar com dados padronizados
tree_pad.fit(X_train_pca_pad, y_train)

# Treinar com dados não padronizados
tree_no_pad.fit(X_train_pca_no_pad, y_train)

# 3. Avaliar acurácia
# Dados padronizados
y_train_pred_pad = tree_pad.predict(X_train_pca_pad)
y_test_pred_pad = tree_pad.predict(X_test_pca_pad)
acc_train_pad = accuracy_score(y_train, y_train_pred_pad)
acc_test_pad = accuracy_score(y_test, y_test_pred_pad)

# Dados não padronizados
y_train_pred_no_pad = tree_no_pad.predict(X_train_pca_no_pad)
y_test_pred_no_pad = tree_no_pad.predict(X_test_pca_no_pad)
acc_train_no_pad = accuracy_score(y_train, y_train_pred_no_pad)
acc_test_no_pad = accuracy_score(y_test, y_test_pred_no_pad)

# 4. Comparar resultados
print("Comparação de Acurácia:")
print(f"- Acurácia Treino (Com Padronização): {acc_train_pad:.4f}")
print(f"- Acurácia Teste (Com Padronização): {acc_test_pad:.4f}")
print(f"- Acurácia Treino (Sem Padronização): {acc_train_no_pad:.4f}")
print(f"- Acurácia Teste (Sem Padronização): {acc_test_no_pad:.4f}")


Comparação de Acurácia:
- Acurácia Treino (Com Padronização): 0.8587
- Acurácia Teste (Com Padronização): 0.7743
- Acurácia Treino (Sem Padronização): 0.8912
- Acurácia Teste (Sem Padronização): 0.8222


### Comparação dos Resultados

A análise dos resultados mostra que a **acurácia da árvore de decisão sem padronização** é levemente superior tanto na base de treino quanto na base de teste, indicando que os dados originais podem conter informações relevantes que se perdem ao padronizar. Por outro lado, a **acurácia na base de treino é menor nos dados padronizados**, sugerindo que a padronização ajuda a evitar overfitting ao reduzir o impacto de variáveis com escalas maiores. Apesar disso, a diferença de desempenho na base de teste é pequena, indicando que ambas as abordagens podem ser viáveis dependendo do objetivo do modelo.
